[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NNHieu/TrueAnswer_ZaloAIChallenge2019_VietnameseWikiQA/blob/master/PlayWithData.ipynb)

## Import lib và set dir

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import pandas as pd
import os
import json
import importlib

### Nếu chạy bằng Colab nhớ điền đường dãn đến thư mục Dataset ở drive

In [5]:
if 'google.colab' in str(get_ipython()):
    print("Đang chạy bằng google colab")
    
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)
    import os
    os.chdir("drive/My Drive/<đường dãn đến Project>")
else:
    print("Đang không chạy bằng google colab")
    # os.chdir("./Dataset")
DATASET_PATH = 'Dataset'

Đang không chạy bằng google colab


In [ ]:
from textprocessor.text_transform import ZaloTextPreprocess

## Mở các file train.json và test.json

In [6]:
importlib.reload(zd)

zltp = zd.ZaloTextPreprocess()
zltp.load_data(DATASET_PATH)

### Sơ qua dữ liệu dùng cho training:
Tên file: train.json

Bao gồm ~18K item đã được gán nhãn. Mỗi item bao gồm:

* **id**: id của mỗi item
* **question**: câu hỏi
* **text**: đoạn text `liên quan` đến câu hỏi, kết quả từ 1 dummy search tool. Là 1 đoạn Wikipedia article.
* **title**: title của Wikipedia article mà đoạn text được lấy từ.
* **label**: nhãn, 2 loại, `True` hoặc `False`, đoạn text trên `có trả lời cho câu hỏi` hoặc `không trả lời được`.

Như ở dưới thì câu hỏi `Quang Hải giành được chức vô địch U21 quốc gia năm bao nhiêu tuổi` có thể xuất hiện nhiều lần, nhưng đi kèm với đoạn text và label tương ứng khác nhau.

In [7]:
# Read Training Data
train_df = zltp.train_df
print('Training data shape: ',train_df.shape)
train_df.head(5)

Training data shape:  (18108, 5)


,id,question,title,text,label
0,u7-1570446247_1,Quang Hải giành được chức vô địch U21 quốc gia...,Nguyễn Quang Hải (sinh 1997),"Năm 2013 , Nguyễn Quang Hải giành chức vô địch...",True
1,u7-1570446247_2,Quang Hải giành được chức vô địch U21 quốc gia...,Nguyễn Quang Hải (sinh 1997),"Sau chức vô địch U-21 quốc gia 2013 , Nguyễn Q...",True
2,u7-1570446247_0,Quang Hải giành được chức vô địch U21 quốc gia...,Nguyễn Quang Hải (sinh 1997),Anh bắt đầu gia nhập lò đào tạo trẻ Hà Nội T&T...,False
3,u7-1570446247_3,Quang Hải giành được chức vô địch U21 quốc gia...,Nguyễn Quang Hải (sinh 1997),"Năm 2011 , Nguyễn Quang Hải mới 14 tuổi được g...",False
4,u7-1570445661_0,Mỗi hiệp bóng đá kéo dài bao lâu,Bóng đá,Một trận đấu bóng đá thông thường có hai hiệp ...,True


### Sơ qua dữ liệu dùng cho testing:
Tên file: test.json

Bao gồm các test cases có trong tập public test. Mỗi test case bao gồm:

* **__id__**: test case ID
* **question**: câu hỏi
* **paragraphs**: **các** đoạn văn có được từ 1 dummy search tool. Mỗi đoạn có 1 ID riêng **trong test case** (nghĩa là các đoạn trong các test case khác nhau có thể có cùng id). Các đội tham gia phải tìm ID của các đoạn có thể trả lời cho `câu hỏi`.
* **title**: title của Wikipedia article mà đoạn text được lấy từ.

In [8]:
# Read Test Data
test_df = pd.read_json('Dataset/test.json', orient='records')
print('Test data shape: ',test_df.shape)
test_df.head(5)

Test data shape:  (501, 4)


,__id__,question,title,paragraphs
0,test_0001,Toà nhà nào cao nhất Việt Nam,The Landmark 81,"[{'id': 'p1', 'text': 'The Landmark 81 là một ..."
1,test_0002,Messi thuận chân nào,Lionel Messi,"[{'id': 'p1', 'text': 'Đôi chân ngắn nhưng mạn..."
2,test_0003,Sao Hoả còn được gọi là gì,Sao Hỏa,"[{'id': 'p1', 'text': 'Sao Hoả còn gọi là : Ho..."
3,test_0004,Hồ Hoàn Kiếm nằm ở đâu,Hồ Hoàn Kiếm,"[{'id': 'p1', 'text': 'Hồ Hoàn Kiếm ( chữ Nôm ..."
4,test_0005,Lionel Messi sinh ra ở đâu,Lionel Messi,"[{'id': 'p1', 'text': 'Lionel Andrés "" Leo "" M..."


Để cho tiện, chuyển test set về cùng dạng với training set

In [9]:
test_df = zltp.test_df
test_df.head(5)

,id,question,title,text
0,test_0001_p1,Toà nhà nào cao nhất Việt Nam,The Landmark 81,The Landmark 81 là một toà nhà chọc trời trong...
1,test_0001_p2,Toà nhà nào cao nhất Việt Nam,The Landmark 81,"Toà tháp cao 81 tầng , hiện tại là toà nhà cao..."
2,test_0002_p1,Messi thuận chân nào,Lionel Messi,Đôi chân ngắn nhưng mạnh mẽ giúp Messi dễ dàng...
3,test_0002_p2,Messi thuận chân nào,Lionel Messi,"Cựu huấn luyện viên Barcelona , Pep Guardiola ..."
4,test_0002_p3,Messi thuận chân nào,Lionel Messi,Ngoài ra anh còn một chân sút phạt chính và ha...


## Tìm hiều sâu hơn
### 1. Basic Data Exploration
#### 1.1 Thông tin cơ bản

In [10]:
# Null values and Data types
print('------------- Train Set -------------')
print(train_df.info())
print('------------- Test Set -------------')
print(test_df.info())

------------- Train Set -------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18108 entries, 0 to 18107
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        18108 non-null  object
 1   question  18108 non-null  object
 2   title     18108 non-null  object
 3   text      18108 non-null  object
 4   label     18108 non-null  bool  
dtypes: bool(1), object(4)
memory usage: 583.7+ KB
None
------------- Test Set -------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2678 entries, 0 to 2677
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        2678 non-null   object
 1   question  2678 non-null   object
 2   title     2678 non-null   object
 3   text      2678 non-null   object
dtypes: object(4)
memory usage: 83.8+ KB
None


Có bị chỗ nào thiếu dữ liệu không ?

In [11]:
train_df.isnull().sum()

id          0
question    0
title       0
text        0
label       0
dtype: int64

In [12]:
test_df.isnull().sum()

id          0
question    0
title       0
text        0
dtype: int64

Không!!

In [13]:
print("Tỷ lệ Label trong trainning set")
train_df.groupby(['label']).size().reset_index().rename(columns={0: "count"})

Tỷ lệ Label trong trainning set


,label,count
0,False,12370
1,True,5738


#### 1.2 Các câu hỏi

In [14]:
print("Có tổng cộng", train_df['question'].value_counts().shape[0], "câu hỏi trong training set")

Có tổng cộng 5070 câu hỏi trong training set


In [15]:
print("Có tổng cộng", test_df['question'].value_counts().shape[0], "câu hỏi trong test set")

Có tổng cộng 501 câu hỏi trong test set


In [16]:
train_questions = set(train_df['question'].unique())
test_questions = set(test_df['question'].unique())
print("Các câu hỏi xuất hiện ở cả training set và test set")
train_questions.intersection(test_questions)


Các câu hỏi xuất hiện ở cả training set và test set


{'Bác Hồ ra đi tìm đường cứu nước năm bao nhiêu',
 'Kinh tế Việt Nam hiện nay đứng hàng thứ mấy trên thế giới',
 'Trụ sở chính của Liên Hiệp Quốc đặt ở đâu',
 'Vịnh Hạ Long nằm ở khu vực nào'}

In [17]:
print("Trong training set")
tmp_df = train_df.groupby(['question', 'title']).size()
# tmp_df['q_len'] = train_df.question.map(lambda x: len(x.split(" ")))
tmp2 = tmp_df.groupby(['question']).size().reset_index().rename(columns={0:'count'})
print("Số cặp (question, title):", tmp_df.shape[0])
print("Một câu hỏi có tối đa", tmp2.max()['count'], "title, và là câu")
train_df.loc[train_df['question'] == 'Quê hương của tổng thống Obama ở đâu'].head(5)

Trong training set
Số cặp (question, title): 15048
Một câu hỏi có tối đa 15 title, và là câu


,id,question,title,text,label
3948,u0-1568958153_0,Quê hương của tổng thống Obama ở đâu,Barack Obama,Barack Hussein Obama II ( IPA : ; sinh ngày 4 ...,True
3949,u0-1568958153_1,Quê hương của tổng thống Obama ở đâu,Barack Obama,Chào đời ngày 4 tháng 4 năm 1961 tại Bệnh viện...,True
3950,u0-1568958153_8,Quê hương của tổng thống Obama ở đâu,Illinois,Thượng nghị sĩ Hillary Clinton sinh tại Illino...,True
3951,u0-1568958153_9,Quê hương của tổng thống Obama ở đâu,Nyanza ( tỉnh Kenya ),Nằm góc phía Tây Nam đất nước . Đến năm 2009 v...,True
3952,u0-1568958153_17,Quê hương của tổng thống Obama ở đâu,Illinois,"Trong cuộc bầu cử tổng thống Mỹ 2008 , cả hai ...",True


In [22]:
tmp2['q_len'] = tmp2.question.map(lambda x: len(x.split(" ")))
print("Độ dài lớn nhất của câu hỏi là", tmp2['q_len'].max(), "từ, ngắn nhất", tmp2['q_len'].min(), "từ")
pd.concat([tmp2.nlargest(2, 'q_len'), tmp2.nsmallest(3, 'q_len')])

Độ dài lớn nhất của câu hỏi là 32 từ, ngắn nhất 3 từ


,question,count,q_len
1810,"Lê Bích Phương, vận động viên duy nhất giành đ...",3,32
2699,Phóng viên chiến trường nào của hãng thông tấn...,2,32
27,AIDS là gì,5,3
30,ASCII là gì,3,3
312,Anime là gì,2,3


In [28]:
tmp2 = train_df['text']
tmp2['text_len'] = tmp2.text.map(lambda x: len(x.split(" ")))
print("Độ dài lớn nhất của câu trả lời là", tmp2['text_len'].max(), "từ, ngắn nhất", tmp2['text_len'].min(), "từ")
pd.concat([tmp2.nlargest(2, 'text_len'), tmp2.nsmallest(3, 'text_len')])

Độ dài lớn nhất của câu trả lời là 594 từ, ngắn nhất 1 từ


,id,question,title,text,label,text_len
17261,u8-1554267633_3,Cục trưởng Cục chính trị - Hậu cần hiện nay là ai,Đỗ Hoàng Anh,Đỗ Hoàng Anh là Á hậu 2 Hoa Hậu Việt Nam 2012 ...,False,594
15976,u8-1554886940_3,Từ khi thành lập đến nước ta có bao nhiêu loại...,Tiền,"Ngày nay, chúng ta sử dụng cả tiền xu và tiền ...",False,497
5812,u5-1562082929_1,Ai là người đã phát minh ra thuyết lượng tử,Thuyết lượng tử năng lượng,formula_7,False,1
5813,u5-1562082929_2,Ai là người đã phát minh ra thuyết lượng tử,Thuyết lượng tử năng lượng,formula_8,False,1
7727,u5-1558706130_2,Bóng đá nam Hàn Quốc mấy lần vô địch tại đấu t...,Giải vô địch bóng đá U-16 thế giới 1989,Asia,False,1


In [ ]:
tmp2['text_len'].value_counts(bin=10)

In [40]:
print("Trong test set")
tmp_df = test_df.groupby(['question', 'title']).size()
# tmp_df['q_len'] = train_df.question.map(lambda x: len(x.split(" ")))
tmp2 = tmp_df.groupby(['question']).size().reset_index().rename(columns={0:'count'})
print("Số cặp (question, title):", tmp_df.shape[0])
print("Một câu hỏi có tối đa", tmp2.max()['count'], "title")
tmp2['q_len'] = tmp2.question.map(lambda x: len(x.split(" ")))
print("Độ dài lớn nhất của câu hỏi là", tmp2['q_len'].max(), "từ, ngắn nhất", tmp2['q_len'].min(), "từ")
pd.concat([tmp2.nlargest(2, 'q_len'), tmp2.nsmallest(3, 'q_len')])

Trong test set
Số cặp (question, title): 501
Một câu hỏi có tối đa 1 title
Độ dài lớn nhất của câu hỏi là 21 từ, ngắn nhất 4 từ


,question,count,q_len
136,Hòn đảo là nơi duy nhất mà chủ tịch Hồ Chí Min...,1,21
232,Mỹ dỡ bỏ lệnh dỡ bỏ hoàn toàn cấm vận vũ khí á...,1,21
146,Khi nào phạt đền,1,4
172,Lionel Messi là ai,1,4
206,Messi thuận chân nào,1,4


In [30]:
tmp2 = test_df[['text']]
tmp2['text_len'] = tmp2.text.map(lambda x: len(x.split(" ")))
print("Độ dài lớn nhất của câu trả lời là", tmp2['text_len'].max(), "từ, ngắn nhất", tmp2['text_len'].min(), "từ")
pd.concat([tmp2.nlargest(2, 'text_len'), tmp2.nsmallest(3, 'text_len')])

Độ dài lớn nhất của câu trả lời là 216 từ, ngắn nhất 16 từ
<ipython-input-30-e1bfec7f475a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp2['text_len'] = tmp2.text.map(lambda x: len(x.split(" ")))


,text,text_len
2403,Trong số các nước đã thiết lập quan hệ ngoại g...,216
1309,Thành phố Cần Thơ trực thuộc trung ương có diệ...,213
1872,Có bộ chiêng chỉ được đánh khi có vật hiến sin...,16
830,Nhà hàng và sky bar ở tầng 66 và 67 có góc nhì...,17
910,World Cup 2010 khép lại với Argentina sau trận...,17
